# Example 6-b

In this example, all input data is given directly in the notebook cells below.  The data is given
in CSV form precisely as would be given in data files.  For each table started by the cell
magic `%%Table`, the table name follows immediately.  If the data was instead provided in a data file
the file name would be the table name + `.csv`.

This example is statically determinate for ease of checking -- the rightmost figure shows
the reactions corresponding to factored values of the loads shown in the left figure.
Units are $N$ and $mm$.

## Test Frame

In [1]:
from IPython import display
display.SVG('data/frame-6b.d/frame-6b.svg')

In [2]:
from Frame2D import Frame2D

f6b = Frame2D('frame-6b')

# Input Data

## Nodes
Table `nodes` (file `nodes.csv`) provides the $x$-$y$ coordinates of each node.  Other columns, such 
as the $z$- coordinate are optional, and ignored if given.

In [3]:
%%Table nodes
NODEID,X,Y,Z
A,0.,0.,5000.
B,0,4000,5000
C,8000,4000,5000
D,8000,0,5000

## Supports
Table `supports` (file `supports.csv`) specifies the support fixity, by indicating the constrained
direction for each node.  There can be 1, 2 or 3 constraints, selected from the set '`FX`', '`FY`' or '`MZ`',
in any order for each constrained node.  Directions not mentioned are 'free' or unconstrained.

In [4]:
%%Table supports
NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY

## Members
Table `members` (file `members.csv`) specifies the member incidences.  For each member, specify
the id of the nodes at the 'j-' and 'k-' ends.  These ends are used to interpret the signs of various values.

In [5]:
%%Table members
MEMBERID,NODEJ,NODEK
AB,A,B
BC,B,C
CD,C,D

## Releases
Table `releases` (file `releases.csv`) is optional and specifies internal force releases in some members.
Currently only moment releases at the 'j-' end ('`MZJ`') and 'k-' end ('`MZK`') are supported.  These specify
that the internal bending moment at those locations are zero.  You can only specify one release per line,
but you can have more than one line for a member.

In [6]:
%%Table releases
MEMBERID,RELEASE
AB,MZK
CD,MZJ

## Properties
Table `properties` (file `properties.csv`) specifies the member properties for each member.
If the '`SST`' library is available, you may specify the size of the member by using the
designation of a shape in the CISC Structural Section Tables.  If either `IX` or `A` is missing,
it is retreived using the `sst` library.  If the values on any line are missing, they
are copied from the line above.

In [7]:
%%Table properties
MEMBERID,SIZE,IX,A
BC,W460x106,,
AB,W310x97,,
CD,,

## Node Loads
Table `node_loads` (file `node_loads.csv`) specifies the forces applied directly to the nodes.
`DIRN` (direction) may be one of `'FX,FY,MZ'`.  'LOAD' is an identifier of the kind of load
being applied and `F` is the value of the load, normally given as a service or specified load.
A later input table will specify load combinations and factors.


In [8]:
%%Table node_loads
LOAD,NODEID,DIRN,F
Wind,B,FX,-200000.

## Support Displacements
Table `support_displacements` (file `support_displacements.csv`) is optional and specifies imposed displacements
of the supports.  `DIRN` (direction) is one of `'DX, DY, RZ'`.  `LOAD` is as for Node Loads, above.

Of course, in this example the frame is statically determinate and so the support displacement
will have no effect on the reactions or member end forces.

In [9]:
%%Table support_displacements
LOAD,NODEID,DIRN,DELTA
Other,A,DY,-10

## Member Loads
Table `member_loads` (file `member_loads.csv`) specifies loads acting on members.  Current
types are `PL` (concentrated transverse, ie point load), `CM` (concentrated moment), `UDL` (uniformly
distributed load over entire span), `LVL` (linearly varying load over a portion of the span) and `PLA` (point load applied parallel to member coincident with centroidal axis).  Values `W1` and `W2` are loads or
load intensities and `A`, `B`, and `C` are dimensions appropriate to the kind of load.

In [10]:
%%Table member_loads
LOAD,MEMBERID,TYPE,W1,W2,A,B,C
Live,BC,UDL,-50,,,,
Live,BC,PL,-200000,,5000

## Load Combinations
Table `load_combinations` (file `load_combinations.csv`) is optional and specifies
factored combinations of loads.  By default, there is always a load combination
called `all` that includes all loads with a factor of 1.0.   A frame solution (see below)
indicates which `CASE` to use.

In [11]:
%%Table load_combinations
CASE,LOAD,FACTOR
One,Live,1.5
One,Wind,1.75
One,Other,2.0

## Solution
The following outputs all tables, prints a description of the input data,
produces a solution for load case '`one`' (all load and case names are case-insensitive)
and finally prints the results.

In [12]:
f6b.input_all()
f6b.print_input()
RS = f6b.solve('one')
f6b.print_results(rs=RS)


Frame frame-6b:


              # of nodal degrees of freedom: 12
  # of constrained nodal degrees of freedom: 5
# of unconstrained nodal degrees of freedom: 7  (= degree of kinematic indeterminacy)

                               # of members: 3
                             # of reactions: 5
                                 # of nodes: 4
                            # of conditions: 2
           degree of statical indeterminacy: 0



Nodes:

Node          X         Y  Constraints  DOF #s
----      -----     -----  -----------  ------
A             0         0  FX,FY,MZ     7,8,9
B             0      4000               0,1,2
C          8000      4000               3,4,5
D          8000         0  FX,FY        10,11,6



Members:

Member   Node-J  Node-K    Length       dcx       dcy  Size                Ix           A  Releases
------   ------  ------    ------   -------   -------  --------      --------       -----  --------
AB       A       B         4000.0   0.00000   1.00000  W310x